In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
from sklearn.model_selection import train_test_split
import numpy as np
from dataclasses import dataclass
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from transformers import AutoConfig, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn


In [ ]:
df = pd.read_csv("/content/summarizdataset.csv")
df.shape

(8378, 4)

In [ ]:
df

,text,type,Processed Text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,culture,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...
...,...,...,...,...
8373,\nتأجل الإضراب العام في قطاع الصحة الذي كان مق...,localnews,تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...,وكانت انعقدت عشية أمس في وزارة الشؤون الاجتماع...
8374,\nكشف الناشطان كريم نوار وعفيف زقية اللذان أشر...,localnews,كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...,وأكدا ان تكلفة العلم لم تتجاوز 3 آلاف دينار مق...
8375,\nتمكّنت فرقة الأبحاث والتفتيش للحرس الوطني بط...,localnews,فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...,كما تمّ إلقاء القبض على عنصر رابع (عمره 32 سنة...
8376,\nقرر الأهالي بمناطق هيشر وعين القارصي والغولا...,localnews,قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...,وتأتي هذه الخطوة على خلفية ما اعتبره أهالي هذه...


In [ ]:
df.drop(columns=['Processed Text'], inplace=True)

In [ ]:
import re

# Cleaning function
def clean_arabic_text(text):
    text = str(text)
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)  # Remove tashkeel
    text = re.sub(r'[a-zA-Z0-9]', '', text)                  # Remove Latin chars & digits
    text = re.sub(r'[^\w\s]', '', text)                      # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()                 # Normalize whitespace
    return text

# Apply cleaning
df['Processed Text'] = df['text'].apply(clean_arabic_text)
df['summarizer'] = df['summarizer'].apply(clean_arabic_text)

In [ ]:
def remove_non_arabic(text):
    return re.sub(r'[A-Za-z0-9@#^&*()+=\[\]{}<>\\|/?!~`:";\']+', '', text)
df['Processed Text'] = df['text'].apply(remove_non_arabic)
df['summarizer'] = df['summarizer'].apply(remove_non_arabic)

In [ ]:
def normalize_arabic(text):
    text = re.sub(r"[إأآا]", "ا", text)
    text = re.sub(r"ى", "ي", text)
    text = re.sub(r"ؤ", "ء", text)
    text = re.sub(r"ئ", "ء", text)
    text = re.sub(r"ة", "ه", text)
    text = re.sub(r"گ", "ك", text)
    return text

df['Processed Text'] = df['text'].apply(normalize_arabic)
df['summarizer'] = df['summarizer'].apply(normalize_arabic)

In [ ]:
# Ensure both columns are strings
df = df[df['text'].apply(lambda x: isinstance(x, str))]
df = df[df['summarizer'].apply(lambda x: isinstance(x, str))]


# Drop old 'text' column and reorder
df.drop(columns=['text'], inplace=True)
df = df[['Processed Text', 'type', 'summarizer']]

# Save cleaned data
df.to_csv('preprocessed_cleaned.csv', index=False)

In [ ]:
df.shape

(8378, 3)

In [ ]:
df = pd.read_csv("/content/preprocessed_cleaned.csv")
df

,Processed Text,type,summarizer
0,\nاشرف رءيس الجمهوريه الباجي قايد السبسي اليوم...,culture,اشرف رءيس الجمهوريه الباجي قايد السبسي اليوم ب...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي الفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الذي الفه باحثون تون...
2,\nاحتضن جناح تونس في القريه الدوليه للافلام بم...,culture,تونس حاضره من جهه اخري ستكون تونس حاضره في قاء...
3,\nشهدت برلين امس الجمعه افتتاح مسجد فريد من نو...,culture,واستاجرت صاحبه المشروع المحاميه والكاتبه سيران...
4,\nنعت وزاره الشّءون الثّقافيّه المنشد الصّوفي ...,culture,نعت وزاره الشءون الثقافيه المنشد الصوفي عز الد...
...,...,...,...
8373,\nتاجل الاضراب العام في قطاع الصحه الذي كان مق...,localnews,وكانت انعقدت عشيه امس في وزاره الشءون الاجتماع...
8374,\nكشف الناشطان كريم نوار وعفيف زقيه اللذان اشر...,localnews,واكدا ان تكلفه العلم لم تتجاوز الاف دينار مقاب...
8375,\nتمكّنت فرقه الابحاث والتفتيش للحرس الوطني بط...,localnews,كما تم القاء القبض علي عنصر رابع عمره سنه يتول...
8376,\nقرر الاهالي بمناطق هيشر وعين القارصي والغولا...,localnews,وتاتي هذه الخطوه علي خلفيه ما اعتبره اهالي هذه...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['Processed Text'], df['summarizer'], test_size=0.2, random_state=42)

In [ ]:
df_sampled = pd.concat([X_train, Y_train], axis=1)
print(df_sampled.shape)

(6702, 2)


In [ ]:
df_sampled = df_sampled.sample(n=1000, random_state=42)
print(df_sampled.shape)

(2000, 2)


In [ ]:
df_sampled.to_csv('/content/summarize_train_final.csv', index=False)

In [ ]:
df = pd.read_csv("/content/summarize_train_final.csv")
df

,Processed Text,summarizer
0,قالت وزيرة الصحة سميرة مرعي فريعة إن وزارتها ت...,قالت وزيرة الصحة سميرة مرعي فريعة إن وزارتها ت...
1,وضعت عملية القرعة التي جرت مساء السبت في مالي ...,وتعد هذه البطولة القارية ترشيحية لنهائيات بطول...
2,قال النائب عن حزب صوت الفلاحين فيصل التبيني إن...,وأضاف في مداخلة له اليوم في برنامج الحدث تونس ...
3,شن تنظيم داعش الارهابي اليوم الأحد هجمات انتحا...,وقال عيسى وقعت تفجيرات بسيارات مفخخة يقودها ان...
4,ألقت الوحدات الأمنية بقفصة مساء الأحد القبض عل...,وقد قاموا بتعنيف العون باستعمال قضبان حديدية و...
...,...,...
995,أكدت وزيرة الصحة سميرة مرعي أنه سيتم فتح باب ا...,وأضافت الوزيرة خلال مداخلة لها اليوم في برنامج...
996,أودعت النقابة الوطنية للصحفيين التونسيين اليوم...,أودعت النقابة الوطنية للصحفيين التونسيين اليوم...
997,تسعى وزارة الهجرة الاسترالية إلى اعتماد نظام ج...,و من المنتظر أن يتم العمل بهذه التقنية خلال شه...
998,جاءت تونس في المرتبة في مؤشر أفضل الدول الذي ت...,جاءت تونس في المرتبة في مؤشر أفضل الدول الذي ت...


In [ ]:
d_sampled = pd.concat([X_test, Y_test], axis=1)
print(d_sampled.shape)

(1676, 2)


In [ ]:
d_sampled = d_sampled.sample(n=273, random_state=42)
print(d_sampled.shape)

(273, 2)


In [ ]:
example_ids = range(0, len(d_sampled))
d_sampled.insert(0, 'example_id', example_ids)
d_sampled

,example_id,Processed Text,summarizer
7121,0,عبر وزير العلاقة مع الهيئات الدستورية والمجتمع...,عبر وزير العلاقة مع الهيئات الدستورية والمجتمع...
3590,1,أعلنت شركة هندية تدعى عن إطلاق أرخص هاتف ذكي ي...,وأطلق على الهاتف الجديد اسم وهو يعمل بنظام أند...
2670,2,أعلن الاتحاد الأوروبي لكرة القدم اليوم عن تشكي...,الوسط بابلو هيرنانديز سيلتا فيجو أندريه هيريرا...
7296,3,أعلنت وزارة التربية اليوم عن فتح مناظرة خارجية...,أعلنت وزارة التربية اليوم عن فتح مناظرة خارجية...
1545,4,ذكرت صحيفة الغارديان البريطانية أن قناة الجزير...,ذكرت صحيفة الغارديان البريطانية أن قناة الجزير...
...,...,...,...
425,268,هاجم نمر حارسة عندما كانت في القفص المخصص له ف...,وأورد موقع سكاي نيوز أن الحادث جد صباح الاثنين...
5507,269,نجح الفريق التونسي المشارك في المسابقة العالمي...,نجح الفريق التونسي المشارك في المسابقة العالمي...
5144,270,تطرقت الكاتبة ألفة يوسف في افتتاحية اليوم الثل...,تطرقت الكاتبة ألفة يوسف في افتتاحية اليوم الثل...
4561,271,تم العثور أمس الأحد على جثة طفلة تبلغ من العمر...,وقد تم نقل الجثة الى المستشفى لعرضها على الطبي...


In [ ]:
d_sampled.to_csv('/content/summarize_test_final.csv',index = False)

In [ ]:
df = pd.read_csv("/content/summarize_test_final.csv")
df

,example_id,Processed Text,summarizer
0,0,عبر وزير العلاقة مع الهيئات الدستورية والمجتمع...,عبر وزير العلاقة مع الهيئات الدستورية والمجتمع...
1,1,أعلنت شركة هندية تدعى عن إطلاق أرخص هاتف ذكي ي...,وأطلق على الهاتف الجديد اسم وهو يعمل بنظام أند...
2,2,أعلن الاتحاد الأوروبي لكرة القدم اليوم عن تشكي...,الوسط بابلو هيرنانديز سيلتا فيجو أندريه هيريرا...
3,3,أعلنت وزارة التربية اليوم عن فتح مناظرة خارجية...,أعلنت وزارة التربية اليوم عن فتح مناظرة خارجية...
4,4,ذكرت صحيفة الغارديان البريطانية أن قناة الجزير...,ذكرت صحيفة الغارديان البريطانية أن قناة الجزير...
...,...,...,...
268,268,هاجم نمر حارسة عندما كانت في القفص المخصص له ف...,وأورد موقع سكاي نيوز أن الحادث جد صباح الاثنين...
269,269,نجح الفريق التونسي المشارك في المسابقة العالمي...,نجح الفريق التونسي المشارك في المسابقة العالمي...
270,270,تطرقت الكاتبة ألفة يوسف في افتتاحية اليوم الثل...,تطرقت الكاتبة ألفة يوسف في افتتاحية اليوم الثل...
271,271,تم العثور أمس الأحد على جثة طفلة تبلغ من العمر...,وقد تم نقل الجثة الى المستشفى لعرضها على الطبي...


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from dataclasses import dataclass

@dataclass
class LabeledExample:
    paragraph: str
    summary: str

class LabeledDataset(Dataset):
    def __init__(self, data: list[LabeledExample]):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained('moussaKam/AraBART')

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx):
        # Unpack the tuple properly (with 2 values: paragraph and summary)
        paragraph, summary = self.data[idx]

        # Ensure paragraph and summary are strings and clean them up
        paragraph = str(paragraph).strip() if isinstance(paragraph, str) else ""
        summary = str(summary).strip() if isinstance(summary, str) else ""

        # Tokenize the paragraph (input)
        inputs = self.tokenizer.encode_plus(
            paragraph,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )

        # Tokenize the summary (labels)
        labels = self.tokenizer.encode_plus(
            summary,
            truncation=True,
            padding='max_length',
            max_length=110,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': labels['input_ids'].flatten()
        }


In [ ]:
# Load and preprocess the labeled dataset
def load_labeled_dataset(file_path, nrows=None):
    df = pd.read_csv(file_path, nrows=nrows)
    dataset = []
    for _, row in df.iterrows():
        paragraph = row['Processed Text']
        summary = row['summarizer']
        dataset.append((paragraph, summary))
    return dataset

In [ ]:
# Load the labeled dataset
labeled_dataset = load_labeled_dataset('/content/summarize_train_final.csv')
dataset = LabeledDataset(labeled_dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
def initialize_model(pretrained_model_name, device):

    model_config = AutoConfig.from_pretrained(pretrained_model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name, config=model_config)
    model.to(device)

    return model

device = 'cuda'
model = initialize_model('moussaKam/AraBART', device)


pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

In [ ]:
def create_data_loader(dataset, batch_size):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

def initialize_training_components(model, learning_rate):
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    return optimizer, criterion


batch_size = 8
learning_rate = 5e-5
data_loader = create_data_loader(dataset, batch_size)
optimizer, criterion = initialize_training_components(model, learning_rate)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
def prepare_validation_dataset(file_path, tokenizer_name='moussaKam/AraBART'):

    df = pd.read_csv(file_path)
    validation_data = [(row['example_id'], row['Processed Text'],row['summarizer']) for _, row in df.iterrows()]

    class ValidationDataset(Dataset):
        def __init__(self, data, tokenizer):
            self.data = data
            self.tokenizer = tokenizer

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
             example_id, paragraph, summarizer = self.data[index]
             inputs = self.tokenizer.encode_plus(paragraph,
                                                 truncation=True,
                                                 padding='max_length',
                                                 max_length=512,
                                                 return_tensors='pt')
             return {
                    'example_id': example_id,
                    'paragraph': paragraph,
                    'summarizer': summarizer,  # Add this line to include the true summary
                    'input_ids': inputs['input_ids'].squeeze(),
                    'attention_mask': inputs['attention_mask'].squeeze()
                   }


    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    dataset_instance = ValidationDataset(validation_data, tokenizer)

    return dataset_instance


file_path = '/content/summarize_test_final.csv'
validation_dataset = prepare_validation_dataset(file_path)


In [ ]:
def create_data_loader(dataset, batch_size, shuffle):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

batch_size = 8
validation_data_loader = create_data_loader(validation_dataset, batch_size, shuffle=False)


In [ ]:
def train_one_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits


        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

def evaluate_model(model, validation_loader, device):
    model.eval()

def save_model(model, directory):
    model.save_pretrained(directory)


num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(model, data_loader, optimizer, device)
    evaluate_model(model, validation_data_loader, device)

save_model(model, 'summrizer_model')


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1, Loss: 1.4565
Epoch 2, Loss: 0.1141
Epoch 3, Loss: 0.0888
Epoch 4, Loss: 0.0763
Epoch 5, Loss: 0.0666
Epoch 6, Loss: 0.0587
Epoch 7, Loss: 0.0500
Epoch 8, Loss: 0.0569
Epoch 9, Loss: 0.0548
Epoch 10, Loss: 0.0420


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
# Load the trained model
model_path = './summrizer_model'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained('moussaKam/AraBART')
model.save_pretrained('/content/summrizer_model')
tokenizer.save_pretrained('/content/summrizer_model')

('/content/summrizer_model/tokenizer_config.json',
 '/content/summrizer_model/special_tokens_map.json',
 '/content/summrizer_model/sentencepiece.bpe.model',
 '/content/summrizer_model/added_tokens.json',
 '/content/summrizer_model/tokenizer.json')

In [ ]:
!zip -r summrizer_model.zip summrizer_model
from google.colab import files
files.download('summrizer_model.zip')

  adding: summrizer_model/ (stored 0%)
  adding: summrizer_model/generation_config.json (deflated 43%)
  adding: summrizer_model/special_tokens_map.json (deflated 52%)
  adding: summrizer_model/tokenizer_config.json (deflated 76%)
  adding: summrizer_model/model.safetensors (deflated 8%)
  adding: summrizer_model/config.json (deflated 60%)
  adding: summrizer_model/tokenizer.json (deflated 77%)
  adding: summrizer_model/sentencepiece.bpe.model (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import csv
import torch

# Function to generate predictions
def generate_predictions(model, data_loader, tokenizer, device):
    model.to(device)
    model.eval()
    predictions = []
    original_word_counts = []

    for batch in data_loader:
        example_ids = batch['example_id'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        paragraphs = batch['paragraph']

        for example_id, inp_id, att_mask, paragraph in zip(example_ids, input_ids, attention_mask, paragraphs):
            paragraph_word_count = len(tokenizer.tokenize(paragraph))
            original_word_counts.append(paragraph_word_count)

            if inp_id.dim() == 1:
                inp_id = inp_id.unsqueeze(0)

            target_length = max(int(0.47 * paragraph_word_count), inp_id.size(1) + 1)
            inp_id = inp_id.to(device)
            att_mask = att_mask.to(device).unsqueeze(0)

            outputs = model.generate(input_ids=inp_id, attention_mask=att_mask, max_length=target_length)
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predictions.append({'example_id': example_id.item(), 'summary': summary})

    return predictions, original_word_counts

def save_predictions_to_csv(predictions, file_path):
    with open(file_path, 'w', encoding='utf-8', newline='') as file:
        fieldnames = ['example_id', 'summary']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for prediction in predictions:
            writer.writerow(prediction)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictions, word_counts = generate_predictions(model, validation_data_loader, tokenizer, device)
save_predictions_to_csv(predictions, 'predictions.csv')

In [ ]:
def calculate_compression_scores(predictions, original_word_counts, target_ratio_lower, target_ratio_upper):
    compression_ratios = []
    scores = []

    for prediction, word_count in zip(predictions, original_word_counts):
        summary = prediction['summary']
        summary_word_count = len(tokenizer.tokenize(summary))
        compression_ratio = summary_word_count / word_count
        compression_ratios.append(compression_ratio)

        if target_ratio_lower <= compression_ratio <= target_ratio_upper:
            scores.append(1.0)
        else:
            score = max(0, (target_ratio_upper - compression_ratio) / (target_ratio_upper - target_ratio_lower))
            scores.append(score)

    return compression_ratios, scores


target_ratio_lower = 0.3
target_ratio_upper = 0.4
compression_ratios, scores = calculate_compression_scores(predictions, word_counts, target_ratio_lower, target_ratio_upper)

for i, (ratio, score) in enumerate(zip(compression_ratios, scores)):
    print(f"Example ID: {predictions[i]['example_id']}, Compression Ratio: {ratio:.2f}, Score: {score:.2f}")


Example ID: 0, Compression Ratio: 0.20, Score: 1.97
Example ID: 1, Compression Ratio: 0.47, Score: 0.00
Example ID: 2, Compression Ratio: 0.35, Score: 1.00
Example ID: 3, Compression Ratio: 0.58, Score: 0.00
Example ID: 4, Compression Ratio: 0.50, Score: 0.00
Example ID: 5, Compression Ratio: 0.15, Score: 2.54
Example ID: 6, Compression Ratio: 0.11, Score: 2.87
Example ID: 7, Compression Ratio: 0.55, Score: 0.00
Example ID: 8, Compression Ratio: 0.24, Score: 1.62
Example ID: 9, Compression Ratio: 1.00, Score: 0.00
Example ID: 10, Compression Ratio: 0.36, Score: 1.00
Example ID: 11, Compression Ratio: 1.00, Score: 0.00
Example ID: 12, Compression Ratio: 0.50, Score: 0.00
Example ID: 13, Compression Ratio: 1.00, Score: 0.00
Example ID: 14, Compression Ratio: 0.16, Score: 2.39
Example ID: 15, Compression Ratio: 0.23, Score: 1.66
Example ID: 16, Compression Ratio: 0.08, Score: 3.20
Example ID: 17, Compression Ratio: 0.66, Score: 0.00
Example ID: 18, Compression Ratio: 0.73, Score: 0.00
Exa

In [ ]:
def summarize_paragraph(paragraph, model, tokenizer, device, ratio=0.47):

    tokens = tokenizer.encode_plus(
        paragraph,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    input_ids, attention_mask = tokens['input_ids'].to(device), tokens['attention_mask'].to(device)

    target_length = max(1, int(ratio * len(tokenizer.tokenize(paragraph))))

    summary_tokens = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=target_length
    )
    summary = tokenizer.decode(summary_tokens[0], skip_special_tokens=True)

    return summary


summary_ratio = 0.50
input_paragraph = "التقنيات الذكية تعزز بشكل كبير التفاعل بين البشر والآلات، وتغير بوضوح نهج الحياة اليومية وطريقة التفكير. فالذكاء الاصطناعي والتعلم الآلي يقدمان حلاً مبتكرًا للتحديات التقنية والاقتصادية، مما يؤدي إلى تطوير العديد من الصناعات وتحسين الخدمات. وبفضل التطور المستمر في هذا المجال، نرى ثورة رقمية تحدث تغييرات عميقة في كافة جوانب الحياة، من العمل إلى الترفيه والتواصل الاجتماعي."
generated_summary = summarize_paragraph(input_paragraph, model, tokenizer, device, summary_ratio)
print("Generated Summary:", generated_summary)


Generated Summary: التقنيات الذكية تعزز بشكل كبير التفاعل بين البشر والآلات، وتغير بوضوح نهج الحياة اليومية وطريقة التفكير. فالذكاء الاصطناعي والتعلم الآلي يقدمان حلا مبتكرا للتحدي


In [ ]:
summary_ratio = 0.50
input_paragraph = "تغير المناخ هو من أبرز التحديات التي تواجه البشرية اليوم. ويُقصد به التغير طويل الأمد في درجات الحرارة وأنماط الطقس على كوكب الأرض، ويرتبط إلى حد كبير بالنشاط البشري، خاصة الانبعاثات الناتجة عن حرق الوقود الأحفوري مثل الفحم والنفط. أدت هذه الانبعاثات إلى تراكم غازات الاحتباس الحراري في الغلاف الجوي، مما تسبب في ارتفاع درجة حرارة الأرض. من أبرز آثار تغير المناخ: ذوبان الجليد القطبي، وارتفاع مستوى سطح البحر، وتغير نمط الأمطار، وزيادة الظواهر الجوية المتطرفة كالفيضانات والجفاف. وللتصدي لهذا التحدي، تتعاون الدول عبر اتفاقيات دولية مثل اتفاقية باريس، وتتبنى سياسات للحد من الانبعاثات، وتحفيز استخدام الطاقة المتجددة، وتعزيز الوعي البيئي بين الأفراد والمجتمعات"
generated_summary = summarize_paragraph(input_paragraph, model, tokenizer, device, summary_ratio)
print("Generated Summary:", generated_summary)


Generated Summary: تغير المناخ هو من أبرز التحديات التي تواجه البشرية اليوم. ويقصد به التغير طويل الأمد في درجات الحرارة وأنماط الطقس على كوكب الأرض، ويرتبط إلى حد كبير بالنشاط البشري، خاصة الانبعاثات الناتجة عن حرق الوقود الأحفوري مثل الفحم والنفط. من أبرز آثار تغير المناخ: ذوبان الجليد القطبي، وارتفاع مستوى سطح


In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu

def calculate_rouge(predictions, true_summaries):
    rouge = Rouge()
    scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}

    for prediction, true_summary in zip(predictions, true_summaries):
        score = rouge.get_scores(prediction['summary'], true_summary)
        scores['rouge-1'].append(score[0]['rouge-1']['f'])
        scores['rouge-2'].append(score[0]['rouge-2']['f'])
        scores['rouge-l'].append(score[0]['rouge-l']['f'])

    avg_scores = {key: sum(value) / len(value) for key, value in scores.items()}
    return avg_scores

def calculate_bleu(predictions, true_summaries):
    bleu_scores = []

    for prediction, true_summary in zip(predictions, true_summaries):
        reference = [true_summary.split()]
        candidate = prediction['summary'].split()
        score = sentence_bleu(reference, candidate)
        bleu_scores.append(score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score

true_summaries = [item['summarizer'] for _, item in enumerate(validation_dataset)]

In [ ]:
# Calculate ROUGE scores
rouge_scores = calculate_rouge(predictions, true_summaries)

# Calculate BLEU score
bleu_score = calculate_bleu(predictions, true_summaries)

# Print all scores
print(f"ROUGE-1 Score: {rouge_scores['rouge-1']:.2f}")
print(f"ROUGE-2 Score: {rouge_scores['rouge-2']:.2f}")
print(f"ROUGE-L Score: {rouge_scores['rouge-l']:.2f}")
print(f"BLEU Score: {bleu_score:.2f}")



ROUGE-1 Score: 0.65
ROUGE-2 Score: 0.60
ROUGE-L Score: 0.60
BLEU Score: 0.50


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
# Load the pretrained model
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained('moussaKam/AraBART')

# Evaluate the pretrained model
pretrained_predictions, _ = generate_predictions(pretrained_model, validation_data_loader, tokenizer, device)


In [ ]:
# Calculate ROUGE and BLEU scores for the pretrained model
pretrained_rouge_scores = calculate_rouge(pretrained_predictions, true_summaries)
pretrained_bleu_score = calculate_bleu(pretrained_predictions, true_summaries)
# Display the scores
print(f"Pretrained ROUGE-1 Score: {pretrained_rouge_scores['rouge-1']:.2f}")
print(f"Pretrained ROUGE-2 Score: {pretrained_rouge_scores['rouge-2']:.2f}")
print(f"Pretrained ROUGE-L Score: {pretrained_rouge_scores['rouge-l']:.2f}")
print(f"Pretrained BLEU Score: {pretrained_bleu_score:.2f}")

Pretrained ROUGE-1 Score: 0.63
Pretrained ROUGE-2 Score: 0.57
Pretrained ROUGE-L Score: 0.55
Pretrained BLEU Score: 0.43
